# CPSC 533V: Assignment 4 - Policy Gradients and Proximal Policy Optimization (PPO)

## 45 points total (9% of final grade)

## Due Date: Fri Oct 29, any-time-on-earth

---

## Submission Information

- Complete the assignment by editing and executing the associated Python files.
- Task 1 should be completed in the notebook, i.e. include your answers under each question.
- Task 2-4 are coding and experiment questions. Copy and paste your results (screenshots and logs) in the notebook.  You should also copy completed code into this notebook and paste under the corresponding questions, they should be only a few lines maximum.
- When done, upload the completed Jupyter notebook (ipynb file) on canvas.
- **We recommend working in groups of two**. List your names and student numbers below (if you use a different name on Canvas).

<ul style="list-style-type: none; font-size: 1.2em;">
<li>Name (and student ID): Kim Dinh </li>
<li>Name (and student ID): Alan Milligan </li>
</ul>

*As always, you are encouraged to discuss your ideas and approaches with other students, even if you are not working as a group.*

## Assignment Background

This assignment is on vanilla policy gradients (VPG) methods and Proximal Policy Optimization (PPO).
You will be implementing the loss functions for vanilla policy gradients (VPG), running some experiments on it, and then implementing clipped-PPO policy gradients and its loss function.  The change for PPO is simple and it yields efficient-to-compute stable policy updates, making PPO one of the most widely used DeepRL algorithms today.


Goals:
- To understand policy gradient RL and to implement the relevant training losses, for both discrete and continuous action spaces
- To observe the sensitivity issues that plague vanilla policy gradients
- To understand and implement the PPO clipping objective and to observe how it addresses these issues

External resources:
- [Sutton's Book Chapter 13: Policy Gradients](http://incompleteideas.net/book/the-book-2nd.html)
- [Andrej Karpathy's post on RL in general and policy gradients specifically](http://karpathy.github.io/2016/05/31/rl/)
- [OpenAI's Spinning Up for coverage of policy gradients and PPO](https://spinningup.openai.com/en/latest/)
- [PPO paper](https://arxiv.org/pdf/1707.06347.pdf)
- [Matthew's StackOverflow Post on PPO](https://stackoverflow.com/questions/46422845/what-is-the-way-to-understand-proximal-policy-optimization-algorithm-in-rl/50663200#50663200)

## Task 0: Preliminaries



### Dependencies

In addition to dependencies from past assignments, we will learn to use TensorBoard to view our experiment results. 
```
pip install tensorboard
```

If you want to experiment with LunarLander instead of Cartpole, you'll also need to install the box2d environment.
```
pip install 'gym[box2d]'
```

### Debugging


You can include:  `import ipdb; ipdb.set_trace()` in your code and it will drop you to that point in the code, where you can interact with variables and test out expressions.  We recommend this as an effective method to debug the algorithms.

---

### Quick recap of policy gradients

The idea is that we create a **differentiable policy** $\pi$ to be optimized so as to yield actions that yield high return.  To optimize the policy, we generate samples in the environment and we use those to compute a "modulated gradient" usable for gradient ascent on the policy parameters.  The modulated gradient consists of two terms: (1) the bare policy gradient term $\text{log}(\pi_\theta(a_t | s_t))$,  and the (2) reward/advantage modulator term $A_t$.  Note that $a_t$ is the action that was actually chosen and sent to the environment.  In PyTorch, we implement this modulated gradient by multiplying the two terms together in the following loss function and then calling backward on it:
$$L^{PG}(\theta) = \text{log}(\pi_\theta(a_t | s_t)) * A_t$$

The policy gradient term by itself indicates the direction required to move the policy parameters to *make the action that we chose more probable*.  By itself, this does nothing useful, if applied equally to all samples.  However, by multiplying this gradient by the advantage $A_t$, the full modulated gradient tells us how to move in the direction that makes good actions more probably and bad actions less probable.  When $A_t$ is large in absolute value, we should change the probability a lot. When $A_t$ is negative, we should make that action less likely.  This lets us use a non-differentiable reward signal to modulate the policy's gradient.

Here is a reference of a full vanilla policy gradient algorithm from OpenAI's Spinning Up resources.  This uses a critic value function $V$ trained to predict return.

<img src="vanilla_PG.png" width=800/>

---

## Task 1: Getting up to speed [14pts]
We have provided template code to get started.
For an overview, the files are: `models.py`, `pg_buffer.py`, `main.py`, `utils.py`.  You need only modify `main.py`, but you may modify the others if you so choose.
- `model.py` has the implementation of the networks and the action distributions we will use 
- `pg_buffer.py` has the implementation of the policy gradient buffer (similar to a replay buffer, but only for the most recent on-policy data)
- `main.py` has the (incomplete) implementation of the policy gradient losses and training loop
- `utils.py` utility (helper) functions


### `models.py`

#### 1.a.  Read `models.py` [1pts]
Read through `models.py` and describe what the contained classes do and how they are used.  Include notes that also help support your own understanding and any questions you have.  If you find an anwer to these questions later, you can write them here. Pay attention to the distributions used and how they are parameterized, and also what is different between data collection and optimization time.

<font color='red'>
Answer:
</font>

`Network` defines the architecture for the policy network and the value network. `DiscreteActor` defines the policy network for the case of discrete actions. `GaussianActor` defines the policy network for the case of continuous actions. Finally, we have `ActorCritic`, which will contain a `DiscreteActor` in the case of discrete action space and a `GaussianActor` otherwise. It will also have another network that will act as a value function. With it you can step/act which essentially entails giving it an observation and returning an action and a value and log probability in the case of step or just action in the case of act.

#### 1.b.  Categorical distribution [1pts]
Imagine we have 4 possible actions {0, 1, 2, 3}, and our network outputs logits of `[-2.5, 0.9, 2.4, 3.7]`.  How does `Categorical` convert this into a valid probability distribution, i.e., having probabilities that sum to 1?  What mathematical function is used and what would be the probabilities returned in this case?

<font color='red'>
Answer:
</font>

The probability distrubtion over the actions is computed by applying the softmax function on the network output. 

Let $Z = e^{-2.5} + e^{0.9} + e^{2.4} + e^{3.7}$. Then the probabilites returned in this case is $\frac{1}{Z}[e^{-2.5}, e^{0.9}, e^{2.4}, e^{3.7}]^T \approx [0.00152, 0.04554, 0.20409, 0.74886]^T$

#### 1.c. Gradient of Categorical distribution [3pts]
Continuing from the previous question, assume that we sample from that distribution such that we choose the action corresponding to index 2 (i.e., $a_t = 2$).  Now we want to compute the log prob gradient of this action.  What would be the value of this gradient with respect to all of the logit inputs? In other words, what is $\nabla_{\text{logits}} \text{log}(\pi(a_t))$ if $\pi$ is our Categorical?

You can solve this either by deriving the gradient on paper using your answer from 1.b. or by empirically computing it with code.  In the latter case, you may use the pseudocode below, but you must write a mathematical expression for how the logit gradients are related to the probabilities of the Categorical (`c.probs`).

```
logits = torch.nn.Parameter(torch.tensor([-2.5, 0.9, 2.4, 3.7]))   # imagine these came from the output of the network
c = Categorical(logits=logits)
a_t = torch.tensor(2)  # imagine this came from c.sample()
logp = c.log_prob(a_t)
logp.backward()
print(logits.grad)
```

<font color='red'>
Answer:
</font>

Suppose the number of actions is $n$ and the network outputs logits $x = [x_0, x_1, ..., x_{n-1}]^T$. Let $Z(x) = \sum_{i=0}^{n-1} e^{x_i}$. Then
$$\log(\pi(a_t)) = \log(e^{x_{a_t}} / Z(x)) = x_{a_t} - \log Z(x)$$
So $\nabla_{x_{a_t}}\log(\pi(a_t)) = 1 - e^{x_{a_t}} / Z(x)$ and $\nabla_{x_i}\log(\pi(a_t)) = -e^{x_i} / Z(x)$ for $i\neq a_t$.

In the example above, we have
$$\nabla_{\text{logits}}\log(\pi(2)) = \left[\frac{-e^{-2.5}}{Z}, \frac{-e^{0.9}}{Z}, 1-\frac{e^{2.4}}{Z}, \frac{-e^{3.7}}{Z}\right]^T \approx [-0.00152,\; -0.04554,\; 0.79591,\; -0.74886]^T$$

#### 1.d. Gaussian actor [2pts]
Now imagine we have a continuous action space with 2 actions, and our network outputs a mu of `[0.0, 1.2]`.  Then assume we sampled from that distribution to get $a_t = [0.1, 1.0]$.  What is $\nabla_\mu \text{log}(\pi_\mu(a_t))$ if $\pi$ is our Normal?  Give the value for this case, and write a mathematical expression for the gradient value in general, as a function of $\mu$ and $a_t$.

<font color='red'>
Answer:
</font>

Suppose $\mu, \sigma, a_t\in\mathbb{R}^n$. Then
$$\log(\pi_\mu(a_t)) = \sum_{i=0}^{n-1} \log(\pi_{\mu_i}(a_{t,i})) = \sum_{i=0}^{n-1} \log\left(\frac{1}{\sqrt{2\pi\sigma_i^2}}e^{-(a_{t,i}-\mu_i)^2/(2\sigma_i^2)}\right) = \sum_{i=0}^{n-1} \left(-\frac{(\mu_i-a_{t,i})^2}{2\sigma_i^2} - \frac{1}{2}\log(2\pi\sigma_i^2)\right)$$

We then have $\frac{\partial}{\partial\mu_i}\log(\pi_\mu(a_t)) = (a_{t,i}-\mu_i)/\sigma_i^2$. Thus $\nabla_\mu \log(\pi_\mu(a_t)) = (a_t-\mu) / \sigma^2$ where $\sigma$ is squared element-wise and the division by $\sigma^2$ is element-wise.

In the given example, the gradient value is $[0.1/\sigma_0^2,\; -0.2/\sigma_1^2]^T$

#### 1.e. Meaning of these gradients [1pts]
For both continuous and discrete actions, what are these gradients telling us to do, in terms of the logits and the mus and the actions chosen?  

<font color='red'>
Answer:
</font>

The gradients tell us the directions to change the logits/mus so that the chosen actions will have higher probability of being taken.

###  `pg_buffer.py`

This code implements a buffer used to store the data we acculumate so we can process it in a batch.
Notably, it also computes GAE-Lambda Advantages. To answer the questions below, you should first skim the GAE paper, including at least the abstract and Equation 1 with the different options for $\Psi$ (`psi`): https://arxiv.org/pdf/1506.02438.pdf.  

#### 1.f  Why use GAE-lambda? [1pts]
What is the main argument from the GAE paper about why one should use the Advantage function, (rather than sum of discounted future reward for example) for our choice of $A_t$?

<font color='red'>
Answer:
</font>

Using the Advantage function yields lower variance than other choices and reduces the number of samples needed.

#### 1.g  Paper to code correspondence [1pts]
See the `finish_path` function.  In which line of the GAE algorithm (pg 8) would you call it? And which equation in the GAE paper does the `adv_buf` line (`pg_buffer.py:61`) correspond to?

<font color='red'>
Answer:
</font>

The `finish_path` function is called on line 4 of the GAE algorithm (it actually covers line 4 and 5 of the algorithm). The `adv_buf` corresponds to equation (16) in the paper.

### 1.3 `main.py`

#### 1.h. Read `main.py` [2pts]

Read through the code and write down any notes that help your understanding of what is going on, as well as any questions you have.

<font color='red'>
Answer:
</font>

What is `approx_kl` in the function `compute_loss_pi`?

One update call involves `train_pi_iters` steps of gradient ascent for policy network, and `train_v_iters` steps of gradient descent for value network.

#### 1.i. Order of data collection and updating [1pts]
Note the order that we collect data and run optimization in.  How many steps do we collect data before running an update (w/ default args)?  Then how many optimization steps do we run in `update` by default?

<font color='red'>
Answer:
</font>

Number of data collection steps before running an update is `steps_per_epoch` which is 1000 by default. Number of optimization steps for policy network and value network are `train_pi_iters` and `train_v_iters`, which by default are 4 and 40 respectively.

#### 1.j. End of episode handling [1pts]
Describe how the episode terminals / timeouts are handled

<font color='red'>
Answer:
</font>

If we reach `max_ep_len` or `steps_per_epoch` then run `ActorCritic.step` one more time to get the value function at the last state. Otherwise the last state is a terminal state and its value is 0. If an episodes terminates (not because of reaching number of steps per epoch) then save episode length and total reward. Then reset the state.

---

## Task 2: Implementing Policy Gradient Losses [10pts]

Now you will implement the vanilla policy gradient losses.  This includes the policy gradient loss $L^{PG}$ as well as a critic loss $L^{V}$, where the critic will be used to compute better advantages. You can reference any external sources you would like, but we suggest first trying to implement the losses without these.

$$L^{PG}(\theta) = \text{log}(\pi_\theta(a_t | s_t)) A_t$$

$$L^{V}(\phi) = (V_{\phi}(s_t) - R_t)^2$$

In this homework, choose between CartPole and LunarLander, although experiment with other environments if you are feeling adventurous.  We recommend LunarLander because it is fun and more challenging than CartPole, and good policies are generally quick to learn.  It takes around 10 minutes to reach interesting behavior on a decent computer, and should be fine for this homework.  However, if you find that it is taking too long to train, you can switch to CartPole.  LunarLander also has both discrete and continuous versions so you can try both modes.

- Fill in the TODOs in the `compute_loss_pi` and `compute_loss_v` functions.
- Run your code and make sure it is correct.

The figure below gives examples of the learning curves that you can expect to see with a correct implementation.  This is LunarLander-v2 performance run with the default arg settings.  Note that watching the losses is not as helpful as it is supervised learning. Losses in RL can be deceiving.  They can be increasing, while your policy is still improving a lot.  The reverse can also happen.  They are mostly good to watch as a sanity check and diagnostic. Also note that entropy is a less obvious, but very helpful metric to watch in RL, especially for discrete action spaces.  It should not stay at its maximum, nor should it drop very quickly; it should somewhat gradually decrease as shown in the figure. 

![example curves](./Ut7R1C9.png)
You might see something slightly different due to small differences in your implementation.  Command to run: `tensorboard --log_dir=logs/`

In [1]:
# ANSWERS for Task 2

# Copy your completed functions (or relevant sections) from main.py and paste them here

"""
if args.loss_mode == 'vpg':
    loss_pi = -torch.mean(logp * psi)


loss_v = torch.mean((v - ret.unsqueeze(1)) ** 2)
"""

---

## Task 3: Experimenting with the code [11pts]
 
Once you verify your losses are correct by seeing that your policy starts learning, you will run some experiments.  For this, we have created several command line options that can be used to vary parameters, as well as a logging system that prints to stdout and logs scalars (and optionally gifs) to TensorBoard.  

#### 3.a.  REINFORCE vs. GAE-Lambda [3pts]

As the GAE paper discusses, there are many possible choices for advantage term to use in the policy gradient.  One of the first ones imagined is the discounted future return (`future_return` in the code).  This choice leads to the REINFORCE algorithm (excerpted from the [Sutton book Chapter 13](http://incompleteideas.net/book/the-book-2nd.html) for your reference (where $G$ is discounted future return):

![REINFORCE](./WzyIzgg.png)

You will compare REINFORCE advantage (discounted return) to GAE lambda advantage.  Before you run the experiment, write down what you think will happen.  Why might REINFORCE do better or why might GAE-Lambda do better for this environment? Then run the two following experiments and measure the difference.  You should run them for at least 100 epochs, and maybe more if you want.  Then write down what happened and include a TensorBoard screenshot with both the results.

```
python3 main.py --psi_mode=future_return --prefix=logs/3a/ --epochs=100  # you can make these longer if you want
```

```
python3 main.py --psi_mode=gae --prefix=logs/3a/ --epochs=100
```

<font color='red'>
Answer:
</font>


Prediction: REINFORCE may generally take longer to learn a good policy compared to GAE-Lambda because as suggested GAE-Lambda is more sample efficient.

Observation: After 200 epochs, REINFORCE actually achieved higher rewards more quickly than GAE-Lambda, which did not match our prediction. We think this may be because REINFORCE is unbiased (using real episode returns) whereas GAE-Lambda is biased (using estimation of value function). The problem is still simple enough that REINFORCE can collect future returns quickly and perform better update steps.

- Training log of REINFORCE

![3a_future_return](figs/3a_future_return.png)


- Training log of GAE-Lambda

![3a_gae](figs/3a_gae.png)

#### 3.b.  Running with different numbers of policy training steps / vanilla policy gradient failure [3pts]

One issue of vanilla policy gradient methods is that they are fairly unstable.
In general one cannot run too many update steps for the most recent data because the policy will then overfit to that local data.
It will update too much based on that local gradient estimate and it will eventually cause more harm than good.
Once this happens, it is very difficult to recover.

This is a well known issue that motivated the development of TRPO and PPO, and you are you going to test this issue for yourself. By default, our code only runs 4 policy iterations during each update phase. What happens if you try to run more?  Try the following experiments, include a screenshot and write some thoughts you had about this.  Anything expected or unexpected?  (Note you will rerun these experiments with PPO in a minute)

```
python3 main.py --prefix=logs/3b/ --train_pi_iters=4  --epochs=150 # you can just also keep your results from part a 
```

```
python3 main.py --prefix=logs/3b/ --train_pi_iters=10 --epochs=150  
```

```
python3 main.py --prefix=logs/3b/ --train_pi_iters=20 --epochs=150
```

<font color='red'>
Answer:
</font>



Using 10 policy iterations during an update actually gave a better performance (reached 150 episode rewards after 200 epochs) compared to 4 policy iterations each update (only achieved 100 episode rewards after 200 epochs). It is likely that the unstable effect of using many update steps is not apparent with 10 policy update steps. However that unstability is shown clearly when we use 20 policy iterations during an update; episode rewards hardly improve even after 200 epochs.

- Training log with 10 policy iterations each update

![3b_pi_iter10](figs/3b_pi_iter10.png)

- Training log with 20 policy iterations each update

![3b_pi_iter20](figs/3b_pi_iter20.png)

#### 3.c.  Design your  own experiment(s) [5pts]

NOTE: you can defer this until after implementing the PPO loss if you wish

Now you get to design your own experiments.  Perhaps you are curious about how the learning rate affects things or how a different network would work.  This is your chance to experiment with whatever you think would be interesting to try.
You are free to make any modifications to the code that you would like to run your experiments.

Here is a further list of ideas:
- network arch, activations
- learning rates
- implementing other $\psi$ versions (e.g., #6)
- continuous environment.  comparing how LunarLander does against LunarLanderContinuous
- effect of gamma parameter
- effect of lambda parameter
- how much better performance is if we don't sample from the mean (deterministic forward pass, for evaluation)
- how different random seeds compare
- anything else that you think of

Describe what you are testing, and your predictions about what is going to happen.
Then run the experiment and report the results, including screenshots.



<font color='red'>
Answer:
</font>

We ran PPO on the LunarLanderContinuous to see if the network architecture would be able to adapt to a continuous action space. With PPO with all the same hyperparamters, the network was able to handle a continuous action space, however it did take a few more itterations to get to a decent policy. We tried the same task only changing to VPG with all other hyperparamters the same. In this case, the results look good occasionally but are highly unstable. The performance is not nearly as good as PPO on the same number of itterations, and we think it would probably take many more itterations for an acceptable policy.

PPO:

![3c](figs/3c.png)

VPG:

![3c2](figs/3c2.png)




---

## Task 4: Trying out the PPO clipping objective [10pts]

The following are useful resources for understanding PPO:
- [OpenAI's Spinning Up for coverage of policy gradients and PPO](https://spinningup.openai.com/en/latest/)
- [PPO paper](https://arxiv.org/pdf/1707.06347.pdf)
- [Matthew's StackOverflow Post on PPO](https://stackoverflow.com/questions/46422845/what-is-the-way-to-understand-proximal-policy-optimization-algorithm-in-rl/50663200#50663200)


Now implement the PPO clipped loss objective in the `compute_loss_pi` function. It is a small fix (only a few lines) to our policy gradients implementation.  After you see that it is learning, by running the command below, you will then compare it to VPG.
```
python3 main.py --loss_mode=ppo
```

This would have been problematic before, but now the algorithm should stay fairly stable:
```
python3 main.py --loss_mode=ppo --prefix=logs/4/ --train_pi_iters=20 --epochs=150
```
vs.

```
python3 main.py --loss_mode=vpg --prefix=logs/4/ --train_pi_iters=20 --epochs=150
```


Record the results of what happened and consider including some screenshots.  You are free to run and include any other tests that you found interesting.  You can also try to further tune PPO and find hyperparameters that make it work better.

<font color='red'>
Answer:
</font>

The code for PPO is below. We ran the above two commands and saw the first set of plots for PPO and second set for VPG. PPO is fairly stable, where as VPG initially looked like it was getting somewhere then completely diverged and was unstable. This is probably due to the number of iterations being high and VPG not being able to handle the instability.

We also tried to run PPO with just 4 policy iterations (default setting). The rewards increased more steadily and it achieved positive rewards more quickly than using 20 policy iterations. So we think even with PPO, it's still a bad practice to use a high number of policy iterations in one update.

PPO with `train_pi_iters=20`:

![4a](figs/4a.png)

VPG with `train_pi_iters=20`:

![4b](figs/4b.png)

PPO with `train_pi_iters=4`:

![4c](figs/4c.png)


In [ ]:

"""
elif args.loss_mode == 'ppo':
    r = torch.exp(logp - logp_old)
    r_clipped = torch.clamp(r, 1-args.clip_ratio, 1+args.clip_ratio)
    loss_pi = -torch.mean(torch.minimum(r*psi, r_clipped*psi))
"""